# Instalar librerías requeridas

In [ ]:
# Instalar rdflib y sparqlwrapper
!pip install rdflib sparqlwrapper google-generativeai

In [36]:
# Importar las bibliotecas necesarias
from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery
import requests
from google.colab import userdata
import google.generativeai as genai

# Endpoint Data.World
endpoint_url = "https://query.data.world/sparql/acabreramti/farmacias-chile"


In [62]:
data_world_token = userdata.get('DATA_WORLD_API_KEY')
gemini_api_key = userdata.get('GOOGLE_API_KEY_1')

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

In [63]:
# Definir la consulta SPARQL
sparql_query = """
    PREFIX ex: <https://midas.minsal.cl/>
    PREFIX schema: <https://schema.org/>


      SELECT ?comuna (COUNT(?farmacia) AS ?farmacias)
      WHERE {
        ?farmacia a schema:Pharmacy ;
                  ex:comuna ?comuna .
      }
    GROUP BY ?comuna
    ORDER BY DESC(?farmacias)
"""

In [50]:
def query_with_token(endpoint, query, token):
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/sparql-query",
        "Accept": "application/sparql-results+json"
    }

    response = requests.post(endpoint, headers=headers, data=query)
    response.raise_for_status()

    return response.json()

In [59]:
def get_data_from_dataworld(sparql_query):
    try:
      results_json = query_with_token(endpoint_url, sparql_query, data_world_token)
    except requests.exceptions.RequestException as e:
      print(f"Error al ejecutar la consulta en Data.world: {e}")
      return None

    data_list = []

    if results_json and "results" in results_json and "bindings" in results_json["results"]:
      for row in results_json["results"]["bindings"]:
          data_list.append(row)

    return data_list

def get_gemini_response_with_dataworld(user_query, sparql_query):
    data_from_dataworld = get_data_from_dataworld(sparql_query)

    context = ""
    if data_from_dataworld:
      context += "Aquí hay algunos datos de Data.world que pueden ser relevantes para tu pregunta:\n"
      for row in data_from_dataworld:
        context += f"{row}\n"
    else:
      context += "No se encontraron datos relevantes en Data.world.\n"

    prompt = f"Usuario: {user_query}\n\nContexto:\n{context}\n\nRespuesta de Gemini:"

    try:
      response = model.generate_content(prompt)
      return response.text
    except Exception as e:
      return f"Error al obtener la respuesta de Gemini: {e}"

In [64]:
system = """
    Antes de responder la pregunta, analiza el JSON que se adjunta como contexto.
    Este json es la respuesta de un endpoint de data.world a una consulta en SparQL.
    Analízalo adecuadamente.
    Luego, si no hay informacion relevante en el JSON, busca en tus propios datos para tratar de responder.
    La pregunta es:
    """
user_query = "Cuantas farmacias hay en la comuna de Talcahuano?"
gemini_response = get_gemini_response_with_dataworld(system+user_query, sparql_query)
print(f"Respuesta de Gemini:\n{gemini_response}")

Respuesta de Gemini:
El JSON proporcionado contiene información sobre la cantidad de farmacias en varias comunas. Tras analizar el JSON, se encontró que **en la comuna de Talcahuano hay 31 farmacias**.

